# Mission 2

## Team. Take Out

# wandb 없이 validation만 다루는 코드는 아래에 있으니 참고 부탁드립니다.(markdown에 'for validation(no wandb)' 이라고 써져있는 부분부터 실행시키시면 됩니다.)



# Train

### install libraries

In [1]:
!pip install wandb torchmetrics

### Import wandb and login

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: takeout. Use `wandb login --relogin` to force relogin


True

### Google drive mount

In [3]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
%cd /content/drive/MyDrive/데이터셋
%mkdir save_model
%ls

### Import libraries

In [5]:
from torchmetrics.classification import MulticlassConfusionMatrix
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import unicodedata
import seaborn as sns
from glob import glob

from tqdm.notebook import tqdm
from warnings import filterwarnings
filterwarnings('ignore')

import torch
from torch import nn, optim
import torchvision.transforms.functional as F

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
#from torch.utils.tensorboard import SummaryWriter#
from torchvision import models
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import time

from torch.backends import cudnn
cudnn.benchmark = True

### Set matplotlib font

In [6]:
!apt-get update -qq         # apt-get 패키지 설치 명령어, -qq : 에러외 메세지 숨기기
!apt-get install fonts-nanum* -qq #나눔글꼴 설치

fe = fm.FontEntry(fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumGothic') #파일 저장되어있는 경로와 이름 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) #폰트설정

plt.rcParams['axes.unicode_minus'] =False

### Define functions for training

In [7]:
def seed_all(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
#   torch.cuda.manual_seed_all(seed)
#   torch.backends.cudnn.deterministic = True
#   torch.backends.cudnn.benchmark = False
#   torch.backends.cudnn.enabled = False


def image_folder(LOCAL,nor, cate='train', pad=False):
  if nor == 'cus':
        meanR, meanG, meanB = 0.583938, 0.51390076, 0.4227201
        stdR, stdG, stdB = 0.22708978, 0.2373795, 0.25375083
  elif nor=='all':
    meanR, meanG,meanB = 0.485,0.456,0.406
    stdR, stdG, stdB = 0.229, 0.224, 0.225


#     transforms.RandomChoice([transforms.Compose([SquarePad(),
#                                                                                        transforms.Resize((224,224))]),
#                                                                    transforms.Compose([transforms.Resize(224),
#                                                                                       transforms.CenterCrop((224,224))]),
#                                                                    transforms.Resize((224,224))
#                                                                    ]),
  if cate == 'train':
    data_transform = transforms.Compose([transforms.Resize((224,224)),
                                          transforms.RandomVerticalFlip(p=0.5),#0.5의 확률로 상하반전
                                          transforms.RandomHorizontalFlip(p=0.5),#0.5의 확률로 좌우반전
                                          transforms.RandomRotation(22.5),
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean = [meanR, meanG, meanB],std = [stdR, stdG, stdB])])

  else:
    data_transform = transforms.Compose([transforms.Resize((224,224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize(mean = [meanR, meanG, meanB], std = [stdR, stdG, stdB])])

  data = ImageFolder(LOCAL,transform=data_transform)
  return data


def load_data(data, batch_size,shuffle=False, num=8):
  data_loader = DataLoader(data, batch_size=batch_size,
                            shuffle=shuffle, num_workers=num)
  return data_loader


def load_model(arch, device, length, PATH=None):
  if arch == 18:
    resnet = models.resnet18(pretrained=False)
  elif arch == 34:
    resnet = models.resnet34(pretrained=False)
  elif arch == 50:
    resnet = models.resnet50(pretrained=False)
  elif arch == 101:
    resnet = models.resnet101(pretrained=False)
  elif arch == 152:
    resnet = models.resnet152(pretrained=False)

  fc_in_features = resnet.fc.in_features
  resnet.fc = nn.Linear(fc_in_features,length)

  if PATH != None:
    resnet.load_state_dict(torch.load(PATH)['model'])

  return resnet


def load_criterion():
  return nn.CrossEntropyLoss()


def load_optimizer(model, kind, lr, wd):
  if kind == "sgd":
    optimizer = optim.SGD(model.parameters(),
                          lr=lr,
                          momentum = 0.9,
                          weight_decay=wd)
  elif kind == "adam":
        optimizer = optim.Adam(model.parameters(),
                               lr=lr,
                               weight_decay=wd)
  elif kind == "adamw":
    optimizer = optim.AdamW(model.parameters(),
                            lr=lr,
                            weight_decay=wd)
  return optimizer


def load_lr_scheduler(sche, optimizer, epoch, min_lr):
  if sche == 'lambdalr':
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.97 ** epoch)
  elif sche == 'coslr':
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=min_lr)
  elif sche == 'cosrestart':
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=min_lr)
  elif sche == 'reduce':
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=4, min_lr=min_lr, verbose=True)
  else:
    scheduler = None
  return scheduler


#for cutmix
def rand_box(size, lam):
  W = size[2]
  H = size[3]
  cut_rat = np.sqrt(1.0-lam)
  cut_w = np.int32(W * cut_rat)
  cut_h = np.int32(H * cut_rat)

  cx = np.random.randint(W)
  cy = np.random.randint(H)

  bbx1 = np.clip(cx - cut_w // 2, 0, W)
  bby1 = np.clip(cy - cut_h // 2, 0, H)
  bbx2 = np.clip(cx + cut_w // 2, 0, W)
  bby2 = np.clip(cy + cut_h // 2, 0, H)

  return bbx1, bby1, bbx2, bby2


def train_function(device, epochs, model, train_loader, train_len, val_loader, val_len, criterion, optimizer, lr_scheduler, max_val_corr = 0,cm=3):
  limit = max_val_corr
  location = None
  model.to(device)
  #start training
  for epoch in tqdm(range(epochs),leave=True):
    model.train()
    training_loss = 0.0
    start_time = time.time()

    print(f'epoch: {epoch+1}')

    #training part
    for inputs, labels in tqdm(train_loader,desc = 'Train',leave = False):
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      if cm==3 or (epoch > epochs-10):
        beta = 0
      else:
        beta = 1.0
      '''
      ref : https://sseunghyuns.github.io/classification/2021/05/25/invasive-pytorch/
      '''
      prod = np.random.random()

      if beta>0:
        if cm==0:
            if prod<1/2: # cutmix 확률
                lam = np.random.beta(beta, beta)

                rand_index = torch.randperm(inputs.size()[0]).to(device)
                target_a = labels
                target_b = labels[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_box(inputs.size(), lam)
                inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
                l = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))

                outputs = model(inputs)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            else:
              outputs = model(inputs)
              loss = criterion(outputs, labels)

        elif cm==1:
            if prod<1/2: # mixup 확률
                lam = np.random.beta(beta,beta)
                rand_index = torch.randperm(inputs.size(0))
                target_a, target_b = labels, labels[rand_index]
                inputs = lam * inputs + (1 - lam) * inputs[rand_index,:]
                outputs = model(inputs)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            else:
              outputs = model(inputs)
              loss = criterion(outputs, labels)

        elif cm==2:
            if prod<1/3: # cutmix 확률
                lam = np.random.beta(beta, beta)
                rand_index = torch.randperm(inputs.size()[0]).to(device)
                target_a = labels
                target_b = labels[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_box(inputs.size(), lam)
                inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
                l = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))

                outputs = model(inputs)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)

            elif prod<2/3:#mixup 확률
                lam = np.random.beta(beta,beta)
                rand_index = torch.randperm(inputs.size(0))
                target_a, target_b = labels, labels[rand_index]
                inputs = lam * inputs + (1 - lam) * inputs[rand_index,:]
                outputs = model(inputs)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)

            else:
              outputs = model(inputs)
              loss = criterion(outputs, labels)
      else:
          outputs = model(inputs)
          loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      training_loss += loss.item()

    #validation part
    with torch.no_grad():
      model.eval()
      running_loss = 0.0
      accuracies = []

      for inputs, labels in tqdm(val_loader,desc = 'Validation',leave = False):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        preds = torch.argmax(outputs, 1)

        running_loss += loss.item()
        accuracies.append(preds == labels)

      accuracy = torch.concat(accuracies).float().mean() * 100
      rate_acc = accuracy.item()
      if lr_scheduler is not None:
        lr_scheduler.step(rate_acc)

      print(f'train_loss: {training_loss:.4f} \t val_loss: {running_loss:.4f}\t val_corr: {rate_acc:.4f}% \t time: {(int(time.time() - start_time)//60)}m {(time.time() - start_time)%60:.0f}s')
      wandb.log({'train_loss':training_loss, 'val_loss':running_loss,'val_corr':rate_acc,'time':time.time() - start_time})

      if rate_acc > max_val_corr:
        if os.path.isfile(f'./save_model/mission2-{max_val_corr:.4f}.pt'):
          os.remove(f'./save_model/mission2-{max_val_corr:.4f}.pt')

        location = f'./save_model/mission2-{rate_acc:.4f}.pt'
        torch.save({'model' : model.state_dict(),
                    'optimizer' : optimizer.state_dict()},
                    f'./save_model/mission2-{rate_acc:.4f}.pt')
        max_val_corr = rate_acc
    print("-"*100)

  if limit == max_val_corr:
    torch.save({'model' : model.state_dict(),
                'optimizer' : optimizer.state_dict()},
                f'./save_model/best_model(mission2-{rate_acc}).pt')
  print(f'best accuray is {max_val_corr:.4f}%')
  print("END")

  return location, max_val_corr


def val_function(device,model,val_loader,label_list):
  model.to(device)

  correct_pred = {classname: 0 for classname in label_list}
  total_pred = {classname: 0 for classname in label_list}
  accuracies = []

  temp = [[0]*len(label_list)]*len(label_list)
  temp_tensor = torch.tensor(temp).to(device)
  metric = MulticlassConfusionMatrix(len(label_list)).to(device)

  with torch.no_grad():
    model.eval()

    for inputs, labels in tqdm(val_loader):
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      preds = torch.argmax(outputs, dim=1)

      accuracies.append(preds == labels)
      temp_tensor += metric(preds, labels)

      for img, label, pred in zip(inputs, labels, preds):
        if label == pred:
          correct_pred[label_list[label]] += 1
        total_pred[label_list[label]] += 1

  accuracy = torch.concat(accuracies).float().mean() * 100
  print(f'채점 점수 : {accuracy.item()}')
  for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname} \t is {accuracy:.2f}%')

  save = temp_tensor.to('cpu')
  metrix = np.array(save)
  plt.figure(figsize = (30, 30))
  for_heatmap = []
  for i in label_list:
    for_heatmap.append(unicodedata.normalize('NFC',i))
  sns.heatmap(metrix, annot=True, cmap='Blues',fmt='g',xticklabels=for_heatmap,yticklabels=for_heatmap)
  plt.xlabel('Predicted')
  plt.ylabel('True')
  plt.show()

### Define main function

In [8]:
def main(config=None):
  with wandb.init(config=config):
    config = wandb.config
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    seed_all(42)
    train_data = image_folder(LOCAL='./train', nor=config.nor, cate='train')

    val_data = image_folder(LOCAL='./val', nor=config.nor, cate='val')
    
    batch_size=128
    if config.arch == 50 or config.arch == 101:
        batch_size=64
    elif config.arch == 152:
        batch_size=32
        
    train_loader = load_data(data=train_data, batch_size=batch_size, shuffle=True,num=32)
    val_loader = load_data(data=val_data, batch_size=batch_size, shuffle=False,num=32)

    #여기서 오류나시면 위에 num의 숫자를 줄여주세요
    for _ in tqdm(train_loader, desc = 'load',leave=False):
      continue
    for _ in tqdm(val_loader, desc = 'load',leave=False):
      continue

    train_loader = load_data(data=train_data, batch_size=batch_size, shuffle=True)
    val_loader = load_data(data=val_data, batch_size=batch_size, shuffle=False)

    label_list=train_data.classes

    model = load_model(arch=config.arch,device=device,length=len(label_list), PATH=config.PATH)
    # arch(18,34,50,101,152)
    #PATH(If you load saved model, write path)
    criterion = load_criterion()

    try:
      optimizer = load_optimizer(model=model, kind=config.kind, lr=config.lr, wd=config.weight_decay)
      optimizer.load_state_dict(torch.load(PATH)['optimizer'])
    except:
      optimizer = load_optimizer(model=model, kind=config.kind, lr=config.lr, wd=config.weight_decay)

    lr_scheduler = load_lr_scheduler(sche=config.scheduler, optimizer=optimizer, epoch=config.epoch, min_lr=config.min_lr)

    PATH,max_val_corr = train_function(device=device,
                          epochs=config.epoch,
                          model=model,
                          train_loader=train_loader,
                          train_len=len(train_data),
                          val_loader=val_loader,
                          val_len=len(val_data),
                          criterion=criterion,
                          optimizer=optimizer,
                          lr_scheduler=lr_scheduler,
                          max_val_corr=60,
                          cm=config.cutmix)


### Hyperparameter tuning

In [9]:
sweep_config = {
    'method': 'grid',
    'name': 'ensemble',
    'metric': {
        'goal': 'maximize',
        'name': 'val_corr'
        },
    'parameters': {
        'kind': {'value': 'adamw'},#'adam', 'adamw', 'sgd'
        'scheduler': {'value':'cosrestart'},#'coslr','lambdalr','reduce','cosrestart',None
        'lr': {'value':0.001},
        'min_lr' : {'value':0.000001},
        'epoch': {'value': 300},#잘 설ㅓㅇ
        'arch': {'values': [18, 34, 50, 101, 152]},# 18,34,50,101,152
        'PATH': {'value': None},#모델 위치
        'cutmix':{'value':2},#0-cutmix, 1-mixup, 2-cutmix & mixup
        'weight_decay':{'value':0.001},#가중치 감쇠(l2 regulization)
        'nor':{'value':'cus'},#'cus' or 'all'
     }
}

# 8
sweep_id = wandb.sweep(sweep_config, project='mission-2')
wandb.agent(sweep_id, main)#count파라미터로 횟수 설정 가능(ex - count=1)
wandb.finish()

Create sweep with ID: w2aa0k1l
Sweep URL: https://wandb.ai/takeout/mission-2/sweeps/w2aa0k1l


wandb: Agent Starting Run: eq6ylwd6 with config:
wandb: 	PATH: None
wandb: 	arch: 18
wandb: 	cutmix: 2
wandb: 	epoch: 300
wandb: 	kind: adamw
wandb: 	lr: 0.001
wandb: 	min_lr: 1e-06
wandb: 	nor: cus
wandb: 	scheduler: cosrestart
wandb: 	weight_decay: 0.001


load:   0%|          | 0/263 [00:00<?, ?it/s]

load:   0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

...

Train:   0%|          | 0/1052 [00:00<?, ?it/s]

Validation:   0%|          | 0/132 [00:00<?, ?it/s]

train_loss: 19.4442 	 val_loss: 102.7685	 val_corr: 84.2729% 	 time: 3m 39s
----------------------------------------------------------------------------------------------------
best accuray is 87.2335%
END


time,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁█▁▁▁▁▁▁▁▅▅
train_loss,█▇▆▆▅▅▅▄▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▁▁
val_corr,▁▂▅▄▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇██▇█████████████████
val_loss,██▅▄▂▄▂▃▂▂▂▂▂▂▂▂▂▄▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
time,218.82434
train_loss,19.44421
val_corr,84.27286
val_loss,102.76855


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet18 = load_model(arch=18,device=device,length=42, PATH='./good_model/mission2-18-88.2757.pt')
resnet34 = load_model(arch=34,device=device,length=42, PATH='./good_model/mission2-34-88.9626.pt')
resnet50 = load_model(arch=50,device=device,length=42, PATH='./good_model/mission2-50-88.1573.pt')
resnet101 = load_model(arch=101,device=device,length=42, PATH='./good_model/mission2-101-86.9019.pt')
resnet152 = load_model(arch=152,device=device,length=42, PATH='./good_model/mission2-152-87.2335.pt')

In [42]:
torch.save({'resnet18' : resnet18.state_dict(),
            'resnet34' : resnet34.state_dict(),
            'resnet50' : resnet50.state_dict(),
            'resnet101' : resnet101.state_dict(),
            'resnet152' : resnet152.state_dict(),
                },f'./save_model/mission2.pt')

# for validation(no wandb)

### You should set only location

In [43]:
#model location(.pt)
PATH_resnet = './save_model/mission2.pt'

#data location
LOCAL = './val'

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/데이터셋
%ls

## 제공해주신 validate_dist.ipynb에 사용된 라이브러리들입니다.

In [44]:
import os
from tqdm.notebook import tqdm
from warnings import filterwarnings
filterwarnings('ignore')

import torch
from torch import nn, optim
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
#from torch.utils.tensorboard import SummaryWriter
from torchvision import models
from torchvision import transforms as T
from torchvision.datasets import ImageFolder

from torch.backends import cudnn
cudnn.benchmark = True

## 18 ~ 152까지 학습했던 모델들을 전부 불러오며 구조는

'mission2.pt' = {   
  'resnet18' : resent18 model,   
  'resnet34' : resnet34 model,   
  'resnet50' : resnet50 model,     
  'resnet101' : resnet101 model,    
  'resnet152' : resnet152 model   
}

로 이루어져 있으며
모든 모델은 torchvision.models에서 불러온 모델 + fc layer(42개의 output)입니다.

In [45]:
def load_model_for_ensemble(arch, device, length, PATH=None):
  if arch == 18:
    resnet = models.resnet18(pretrained=False)
  elif arch == 34:
    resnet = models.resnet34(pretrained=False)
  elif arch == 50:
    resnet = models.resnet50(pretrained=False)
  elif arch == 101:
    resnet = models.resnet101(pretrained=False)
  elif arch == 152:
    resnet = models.resnet152(pretrained=False)

  fc_in_features = resnet.fc.in_features
  resnet.fc = nn.Linear(fc_in_features,length)

  if PATH != None:
    resnet.load_state_dict(torch.load(PATH)[f'resnet{arch}'])
  resnet.to(device)
  return resnet

In [46]:
class EnsembleModel(nn.Module):
    def __init__(self, modelA, modelB, modelC, modelD, modelE):
        super().__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC
        self.modelD = modelD
        self.modelE = modelE

    def forward(self, x):
        x1 = self.modelA(x)
        x2 = self.modelB(x)
#         x3 = self.modelC(x)
#         x4 = self.modelD(x)
#         x5 = self.modelE(x)
        return x1 + x2

### 제공해주신 validate.ipynb에 있는 함수를 그대로 복사했습니다.

In [47]:
@torch.no_grad()
def validate_epoch(model: nn.Module, data_loader: DataLoader, device: torch.device):
    ''' data_loader provides inputs and GTs.
        model receives input images from data_loader and produces logits.
        returns the accuracy of model in percent.
    '''
    model.eval()
    accuracies = []
    for images, labels in tqdm(data_loader, total=len(data_loader), mininterval=1, desc='measuring accuracy'):
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        pred = torch.argmax(logits, dim=1)

        accuracies.append(pred == labels)

    accuracy = torch.concat(accuracies).float().mean() * 100
    return accuracy.item()

meanR, meanG, meanB = 0.583938, 0.51390076, 0.4227201   
stdR, stdG, stdB = 0.22708978, 0.2373795, 0.25375083   
입니다.

In [48]:
meanR, meanG, meanB = 0.583938, 0.51390076, 0.4227201
stdR, stdG, stdB = 0.22708978, 0.2373795, 0.25375083

data_transform = T.Compose([T.Resize((224,224)),
                            T.ToTensor(),
                            T.Normalize(mean = [meanR, meanG, meanB], std = [stdR, stdG, stdB])])

data = ImageFolder(LOCAL,transform=data_transform)

#if error is here, you change num_workers
data_loader = DataLoader(data, batch_size=1,
                            shuffle=False, num_workers=128)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

length = len(data.classes)
resnet18 = load_model_for_ensemble(arch=18,device=device,length=length, PATH=PATH_resnet)
resnet34 = load_model_for_ensemble(arch=34,device=device,length=length, PATH=PATH_resnet)
resnet50 = load_model_for_ensemble(arch=50,device=device,length=length, PATH=PATH_resnet)
resnet101 = load_model_for_ensemble(arch=101,device=device,length=length, PATH=PATH_resnet)
resnet152 = load_model_for_ensemble(arch=152,device=device,length=length, PATH=PATH_resnet)

ensemble_model = EnsembleModel(resnet18,resnet34,resnet50,resnet101,resnet152)
score = validate_epoch(ensemble_model, data_loader, device)
print(score)

measuring accuracy:   0%|          | 0/4222 [00:00<?, ?it/s]

90.21790313720703
